# Reading and Plotting a KAZR Radar File

Let us start we an introductory on what is Py-ART, then after we will begin to code away.

The Python ARM Radar Toolkit or Py-ART:

 * Py-ART is a Python module for plotting, correcting and analyzing weather radar data.
 * Development began to address the needs of ARM with the acquisition of a number of
   new scanning cloud and precipitation radar as part of the American Recovery Act.
 * The project has since expanded to work with a variety of weather radars and a wider user
   base including radar researchers and climate modelers.
 * The software has been released on GitHub as open source software under a BSD license.
   Runs on Linux, OS X. It also runs on Windows with more limited functionality.
   
Contributions from others are always welcome.

What can Py-ART do?
Py-ART can be used for a variety of tasks from basic plotting to more complex
processing pipelines. Specific uses for Py-ART include:

 * Reading radar data in a variety of file formats.
 * Creating plots and visualization of radar data.
 * Correcting radar moments while in antenna coordinates, such as:
    * Doppler unfolding/de-aliasing.
    * Attenuation correction.
    * Phase processing using a Linear Programming method.
 * Mapping data from one or multiple radars onto a Cartesian grid.
 * Performing retrievals.
 * Writing radial and Cartesian data to NetCDF files.

Py-ART essentials links:
 * Landing page, [arm-doe.github.io/pyart/](http://arm-doe.github.io/pyart/)
 * Documentation, [arm-doe.github.io/pyart-docs-travis/](http://arm-doe.github.io/pyart-docs-travis/)
 * Examples, [arm-doe.github.io/pyart/dev/auto_examples/index.html](http://arm-doe.github.io/pyart/dev/auto_examples/index.html)
 * Source Code, [github.com/ARM-DOE/pyart](https://github.com/ARM-DOE/pyart)
 * Mailing list, [groups.google.com/group/pyart-users/](http://groups.google.com/group/pyart-users/)
 * Issue Tracker, [github.com/ARM-DOE/pyart/issues](https://github.com/ARM-DOE/pyart/issues)

Now that we know what Py-ART is, let us began coding.

First we want to import any modules that will be needed.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pyart

When reading in a radar file, we can use the pyart.io.read module or the pyart.aux_io read module.
pyart.io.read can read a variety of different radar formats, such as Cf/Radial, LASSEN and more, while pyart.aux_io expands on other file formats such as ARM KAZR and ODIM H5.
The documentation on what formats can be read by Py-ART can be found here:

http://arm-doe.github.io/pyart-docs-travis/user_reference/io.html

http://arm-doe.github.io/pyart-docs-travis/user_reference/aux_io.html

To read our radar KAZR file:

In [ ]:
radar = pyart.aux_io.read_kazr('./data/sgpkazrgeC1.a1.20110520.000000.cdf')

When reading a radar file, there are a few options to add, such as excluding radar fields. Excluding fields becomes useful when you are loading many radar files at a time and want to conserve memory and save time.

To see the full list of parameters, in a Jupyter Notebook, simply add a question mark after the function.

In [ ]:
pyart.aux_io.read_kazr?

The radar object has a info method. This tells us about the Radar class in general. The info method describes the data in the KAZR file we have just read.

In [ ]:
print(radar.info())

For a more compact view the the radar object info method:

In [ ]:
print(radar.info('compact'))

The number of rays and gates in each ray is also available in the radar object.

In [ ]:
print(radar.nrays)
print(radar.ngates)

Other information about the radar volume is stored in dictionary attributes, with the numerical data stored under the 'data' key.

In [ ]:
print(radar.azimuth)

In [ ]:
print(radar.latitude)  

In [ ]:
print(radar.time)

Some data may not be available in the original file or not applicable to the volume. When this data is missing the attribute is set to: None

In [ ]:
radar.target_scan_rate is None

In the radar object are fields. This is where data such as reflectivity and velocity are stored.
To see what fields are present we can add the fields and keys additions to the variable where the
radar object is stored.

In [ ]:
print(radar.fields.keys())

The fields are stored in a dictionary, each containing coordinates, units and more.
All can be accessed by just adding the fields addition to the radar object variable.
For an individual field, we add a string in brackets after the fields addition to see
the contents of that field.

In [ ]:
print(radar.fields['reflectivity_copol'])

We can go even further in the dictionary and a see within the reflectivity copolar field by accessing the units or data itself.
For example add the string data in brackets after the reflectivity string bracket.

In [ ]:
print(radar.fields['reflectivity_copol']['data'])

In [ ]:
print(radar.fields['reflectivity_copol']['data'][300, 2])

Now that we have loaded the data and inspected it, the next logical thing to do is to visualize the data! Py-ART's visualiation functionality is done through the objects in the pyart.graph module: 

http://arm-doe.github.io/pyart-docs-travis/user_reference/graph.html

In Py-ART there are 5 visualization classes in pyart.graph:

* RadarDisplay	                            
* RadarMapDisplay	                        
* AirborneRadarDisplay	                    
* RadarMapDisplayCartopy                    

Plotting grid data
* GridMapDisplay

For the examples in this tutorial, we will be using RadarDisplay. Lets first see what type of scan type our radar data is.

In [ ]:
print(radar.scan_type)

Now that we know the radar scan type, we can choose which display function to use, in this case vpt. Py-ART also how plotting functions for ppi and rhi.

To display a radar image, we create a figure first and then create a RadarDisplay object from the radar. The following example displays the reflectivity copolar from the radar:

In [ ]:
fig = plt.figure(figsize=[12, 8])
display = pyart.graph.RadarDisplay(radar)
display.plot_vpt('reflectivity_copol',
                 vmin=-8, vmax=64,
                 cmap='pyart_HomeyerRainbow')
plt.show()

You can change many parameters in the graph by changing the arguments to plot_vpt. As you can recall from earlier. simply view these arguments in a Jupyter notebook by typing:

In [ ]:
display.plot_vpt?

Let's display velocity instead of reflectivity, but first let's get an idea of the nyquist velocity to better set our vmin and vmax of our plot. We can do this by accessing the radar object's intrument parameters.

In [ ]:
print(radar.instrument_parameters['nyquist_velocity']['data'])

In [ ]:
fig = plt.figure(figsize=[12, 8])
display = pyart.graph.RadarDisplay(radar)
display.plot_vpt('mean_doppler_velocity_copol',
                 vmin=-6*1.5, vmax=6*1.5)
plt.show()

Py-ART also has the option to use GateFilters. A GateFilter allows for the masking of data points based om certain conditions. For example if we want to mask reflectivity data points when they are above or below a specific threshold, we can do:

In [ ]:
gatefilter = pyart.correct.GateFilter(radar)
gatefilter.exclude_below('reflectivity_copol', 0)
gatefilter.exclude_above('reflectivity_copol', 40)

We can now plot reflectivity copolar again, but this time with the added gatefilter as an argument in the plot_vpt function.

In [ ]:
fig = plt.figure(figsize=[12, 8])
display = pyart.graph.RadarDisplay(radar)
display.plot_vpt('reflectivity_copol',
                 vmin=-8, vmax=64, gatefilter=gatefilter,
                 cmap='pyart_HomeyerRainbow')
plt.show()